<a href="https://colab.research.google.com/github/axgtz/Sistemas-Inteligentes-Team/blob/master/tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Tarea #4: Caníbales y Misioneros </h1>
<h3>Integrantes</h3>
<ul>
<li>Eduardo Badillo Á.: https://github.com/EduBalez/Sistemas-Inteligentes
<li>Roberto Gutierrez : https://github.com/axgtz/Sistemas-Inteligentes

<li>Emilio Hernández: https://github.com/emihdz97/sistemas-inteligentes
</ul>

In [ ]:
!pip install agents

In [ ]:
!pip install utils

In [ ]:
!pip install tensorflow

In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:
from agents import *
from notebook import *
from utils import *

import heapq
import sys
from collections import *


In [ ]:
class Estado:

    def __init__(self,c,m,b=1):
        self.c = c
        self.m = m
        self.b = b

    def setm(self, m):
        self.m = m

    def setc(self, c):
        self.c = c
    
    def setb(self,b):
        self.b = b

    def getm(self):
        return self.m

    def getc(self):
        return self.c
    
    def getb(self):
        return self.b

    def isValid(self):
        return (self.c,self.m) not in [(3,1), (3,2), (2,1), (1,2), (0,1), (0,2)]

    def __repr__(self):
        return "({},{}) b = {}".format(self.c,self.m,self.b)
                                                      
    def __eq__(self, other):
        return self.c == other.c and self.m == other.m and self.b == other.b
    
    def __hash__(self):
        return hash((self.c,self.m,self.b))

    def __lt__(self, other):
        return self.c < other.c and self.m < other.m and self.b < other.b
   

failState = Estado(-1,-1)

In [ ]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass0
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [ ]:
import math

class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        if self.action == None:
            sgn = ''
        elif self.state.getb():
            sgn = '+'
        else:
            sgn = '-'
        return "<Node parent_action: {}{}, state: {},  depth: {}>".format(sgn,self.action, self.state, self.depth)

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def getDepth(self):
        return self.depth

    def __lt__(self, other):
        return self.path_cost < other.path_cost  


failure = Node(failState, path_cost=math.inf) # Indicates an algorithm couldn't find a solution.

In [ ]:
class Cym(Problem):

    initial = Estado(3,3,1)
    goal = Estado(0,0,0)
    huristics = {}
    huristics[initial]=initial
    
    def __init__(self):
        Problem.__init__(self,self.initial,self.goal)

    def actions(self, estado):
        actions = []
        if estado.getb() == 1: # es 1, es decir, va a transportar personas del lado inicial al final
            if estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif estado.c == 1:
                actions.append((1,0))
                if estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        else: # es 0, es decir, va a transportar personas del lado final al inicial 
            if 3-estado.c >= 2:
                actions = actions + [(1,0),(2,0)]
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
            elif 3-estado.c == 1:
                actions.append((1,0))
                if 3-estado.m >= 2:
                    actions = actions + [(0,1),(0,2),(1,1)]
                elif 3-estado.m == 1:
                    actions = actions + [(0,1),(1,1)]
        return actions
    
    def result(self, state, action):
        if state.getb() == 1: # va a restar 
            nuevo = Estado(state.getc()-action[0], state.getm()-action[1], (state.getb()+1)%2)
        else: # va a sumar
            nuevo = Estado(state.getc()+action[0], state.getm()+action[1], (state.getb()+1)%2)
        if nuevo.isValid(): # checar si los canibales no sobrepasan a los misioneros
            return nuevo
        else:
            return failState
        
    def goal_test(self, state):
        return state == self.goal

<h2>Breadth First Search</h2>

In [ ]:
from collections import deque

FIFOQueue = deque

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.goal_test(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in node.expand(problem):
            s = child.state
            if problem.goal_test(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


In [ ]:
problema = Cym()
bfs = breadth_first_search(problema)
for node in bfs.path():
    print(node)

<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>Depth limited search</h2>
<ul>
    <li>Similar to DFS <em>but with a predetermined limit</em></li>
    <li>Solves the drawback of the infinite path in DFS</li>
    <li>The child nodes of the limit nodes will be treated as unsuccesful</li>
    <li>Cutoff value: Defines no solution for the problem within the depth limit</li>
</ul>

In [ ]:
def dls(problem, limit):
    
    def recursive_dls(node, problem, limit):
        if problem.goal_test(node.state): # reached the predefined goal
            return node
        elif limit == 0: # reached the depth limit, no solution was found
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in node.expand(problem):
                result = recursive_dls(child, problem, limit - 1)
                ##print(child, result)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node(problem.initial), problem, limit)

In [ ]:
problema = Cym()
sol = dls(problema, 14)
for node in sol.path():
    print(node)

<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(1, 0), state: (2,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (3,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 4>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 5>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 7>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 8>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 11>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 12>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 13>


<h2>Iterative Deepening Search</h2>
<p>Exactly the same as LDS but varies the depth level until the most optimal is found</p>

In [ ]:
def Iterative_Deepening_Search(problem): # finds fastest backtrack solution
    limit = 0
    horizon = True
    while(horizon):
        r = dls(problem, limit)
        if(type(r) is str):
            limit+=1
        else: horizon = False
    return dls(problem, limit)

In [ ]:
problema = Cym()
sol = Iterative_Deepening_Search(problema)
for node in sol.path():
    print(node)

<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>Best First Graph Search</h2>
<p>Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned.</p>

In [ ]:
def memoize(f, x):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper
    

In [ ]:
class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        try:
            heapq.heappush(self.items, pair)
        except:
            print("Node already in heap")   

    def pop(self):
        """Pop and return the item with min f(item) value."""
        #try:
        return heapq.heappop(self.items)[1] 
        #except:
            #print("Oops!", sys.exc_info()[0], "occurred.")        
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

In [ ]:
def best_first_search(problem, f):

    print(f)

    #f = memoize(f, 'f')

    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        for child in node.expand(problem):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure



In [ ]:
problema = Cym()
#sol = best_first_search(problema,problema.initial)  
print(problema.initial)
print(problema.goal)
sol = best_first_graph_search(problema,lambda node: node.state)  
for node in sol.path():
    print(node)


(3,3) b = 1
(0,0) b = 0
<function <lambda> at 0x7fcd10f229d8>
<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>A* Search</h2>
<p>A* search is best-first graph search with f(n) = g(n)+h(n).</p>

In [ ]:
def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h 
    return best_first_search(problem, lambda n: n.path_cost)

In [ ]:
problema = Cym()
print (problema)
sol = astar_search(problema)

for node in sol.path():
    print(node)


<function astar_search.<locals>.<lambda> at 0x7fcd109cd9d8>
<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>Greedy Best First </h2>
<p>Same as best first search</p>

In [ ]:
# Greedy best-first search is accomplished by specifying f(n) = h(n).
def greedy_best_first_search(problem, h=None):
    return best_first_search(problem,lambda node: node.state)

In [ ]:
problema = Cym()
print (problema)
sol = astar_search(problema) 

for node in sol.path():
    print(node)


<function astar_search.<locals>.<lambda> at 0x7fcd10a41620>
<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>Uniform Cost Search</h2>
<p>Same as A* but without adding functions</p>

In [ ]:
def uniform_cost_search(problem):
    return best_first_search(problem, lambda node: node.path_cost)

In [ ]:
problema = Cym()
print (problema)
sol = uniform_cost_search(problema)

for node in sol.path():
    print(node)


<function uniform_cost_search.<locals>.<lambda> at 0x7fcd109cd598>
<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(2, 0), state: (1,3) b = 0,  depth: 1>
<Node parent_action: +(1, 0), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(1, 0), state: (2,0) b = 1,  depth: 10>
<Node parent_action: -(2, 0), state: (0,0) b = 0,  depth: 11>


<h2>Depth First Limited Search</h2>
<p>Search deepest nodes in the tree first, then through successors</p>

In [ ]:
def depth_first_graph_search(problem):
    frontier = [(Node(problem.initial))]  # Stack

    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and child not in frontier)
    return None

In [ ]:
problema = Cym()
sol = depth_first_graph_search(problema) # TODO h
for node in sol.path():
    print(node)

<Node parent_action: None, state: (3,3) b = 1,  depth: 0>
<Node parent_action: -(1, 1), state: (2,2) b = 0,  depth: 1>
<Node parent_action: +(0, 1), state: (2,3) b = 1,  depth: 2>
<Node parent_action: -(2, 0), state: (0,3) b = 0,  depth: 3>
<Node parent_action: +(1, 0), state: (1,3) b = 1,  depth: 4>
<Node parent_action: -(0, 2), state: (1,1) b = 0,  depth: 5>
<Node parent_action: +(1, 1), state: (2,2) b = 1,  depth: 6>
<Node parent_action: -(0, 2), state: (2,0) b = 0,  depth: 7>
<Node parent_action: +(1, 0), state: (3,0) b = 1,  depth: 8>
<Node parent_action: -(2, 0), state: (1,0) b = 0,  depth: 9>
<Node parent_action: +(0, 1), state: (1,1) b = 1,  depth: 10>
<Node parent_action: -(1, 1), state: (0,0) b = 0,  depth: 11>
